In [59]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

# Read the initial molecule
mol_file_name = '9aRI.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)  # Include hydrogen atoms

# Check the number of atoms in the molecule
num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")

# Define the atoms that define the bond
atom1_index = 0   # Index of the first atom defining the bond (Carbon), adjusted to start from 0
atom2_index = min(num_atoms - 1, 106) # Ensure atom index is within range, adjusted to start from 0

# Define the atoms in each fragment
fragment1_atoms = list(range(0, 17))  # Atom indices of fragment 1, adjusted to start from 0
fragment2_atoms = list(range(17, num_atoms))  # Atom indices of fragment 2

# Calculate the vector along the bond
atom1_coords = mol.GetConformer().GetAtomPosition(atom1_index)
atom2_coords = mol.GetConformer().GetAtomPosition(atom2_index)
bond_vector = np.array(atom2_coords) - np.array(atom1_coords)
bond_length = np.linalg.norm(bond_vector)
unit_bond_vector = bond_vector / bond_length

# User-provided starting distance
starting_distance = 3.100

# Extract the filename without extension
mol_filename = os.path.splitext(mol_file_name)[0]

# Loop to increase the distance along the bond
for step in range(20):
    # Calculate the distance for this step
    distance = starting_distance + 0.5 * (step + 1)

    # Copy the original molecule
    new_mol = Chem.Mol(mol)

    # Calculate the translation vector along the bond
    translation_vector = unit_bond_vector * (distance - bond_length)
    
    # Move fragment 2 along the bond
    for atom_index in fragment2_atoms:
        atom_position = new_mol.GetConformer().GetAtomPosition(atom_index)
        new_position = atom_position + translation_vector
        new_mol.GetConformer().SetAtomPosition(atom_index, new_position)

    # Save the modified molecule coordinates with the original filename as prefix
    output_file_name = f'{mol_filename}_{distance:.3f}.mol'
    Chem.MolToMolFile(new_mol, output_file_name)


Number of atoms in the molecule: 107


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os


### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 


def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 17  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(107, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s


mol_file_name = '9aRI.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=106

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")


num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 


#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 107


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os


### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 17  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9bRI.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=130

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")


num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 


#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 131


In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 17  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9cRI.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=102

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")



num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 



#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 103


In [7]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 18  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen  # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9dRI.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=17 ## Change this to the index of the atom where two scanning bonds share a connecting point (atom)

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")



num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 



#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 95


In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 17  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen  # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9bRBr.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=129 ## Change this to the index of the atom where two scanning bonds share a connecting point (atom)

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")



num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 



#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 131


In [9]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 17  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen  # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9cRBr.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=102 ## Change this to the index of the atom where two scanning bonds share a connecting point (atom)

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")



num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 



#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)


Number of atoms in the molecule: 103


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import os

### This code is used to generate series of structures by increasing distances of two bonds 
### The mol file is generated by Avogadro
### To specify two fragments easily, Coordinates of the first fragment is in the top of the mol file, the second will be at the bottom. Just use Chemcraft to select second fragment and delete it to have the first only.
#### copy coordinates of the first fragment to vsc code, and then do opposite procedure to have coordinate of the second fragment, then copy to the vsc file to have full set of coordinates
#### Halogen is the atom where two scanning bonds share a connecting point (atom)
### Note that this is just for scanning two bonds with a shared atom
# To scan two separate bonds, just change atom indices for defination of fragments and connecting bond between two fragments 

def define_fragments_first_direction(mol):
    num_atoms_f = mol.GetNumAtoms()
    atom1_index_f = 0   # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_f = halogen # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_f = list(range(0, 17)) ## Change this for list of atoms in the first fragment, starting from 0. This case just has 17 atoms upto index 16
    fragment2_atoms_f= list(range(17, num_atoms_f))  
    return atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f


def define_fragments_second_direction(mol):
    num_atoms_s = mol.GetNumAtoms()
    atom1_index_s = 18  # first index of atom 1 to form bond for direction of increasing length 
    atom2_index_s = halogen  # second index of atom 1 to form bond for direction of increasing length 
    fragment1_atoms_s = list(range(0, 17)) + [halogen]  
    fragment2_atoms_s = list(range(17, halogen)) + list(range(halogen, num_atoms_s))  
    return atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s



mol_file_name = '9dRBr.mol'
mol = Chem.MolFromMolFile(mol_file_name, removeHs=False)   
halogen=17 ## Change this to the index of the atom where two scanning bonds share a connecting point (atom)

num_atoms = mol.GetNumAtoms()
print(f"Number of atoms in the molecule: {num_atoms}")



num_loops_first = 11
step_first = 0.100
num_loops_second = 14
step_second = 0.05 



#starting_distance1 = 3.100
#starting_distance2 = 2.700

 
for step1 in range(num_loops_first):

    mol_filename = os.path.splitext(mol_file_name)[0]


    atom1_index_f, atom2_index_f, fragment1_atoms_f, fragment2_atoms_f = define_fragments_first_direction(mol)

    new_mol_f = Chem.Mol(mol)

    # Calculate the vector along the bond
    atom1_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom1_index_f)
    atom2_coords_f = new_mol_f.GetConformer().GetAtomPosition(atom2_index_f)
    bond_vector_f = np.array(atom2_coords_f) - np.array(atom1_coords_f)
    bond_length_f = np.linalg.norm(bond_vector_f)
    unit_bond_vector_f = bond_vector_f / bond_length_f

    starting_distance1 = bond_length_f 
 
    distance1 = starting_distance1 + step_first * step1

    # Calculate the translation vector along the bond
    translation_vector_f = unit_bond_vector_f * abs(distance1 - starting_distance1)

    # Move fragment 1 along the bond
    for atom_index_f in fragment1_atoms_f:
        atom_position_f = new_mol_f.GetConformer().GetAtomPosition(atom_index_f)
        new_position_f = atom_position_f - translation_vector_f
        new_mol_f.GetConformer().SetAtomPosition(atom_index_f, new_position_f)
    
    #Geneate structures for the first round
    #output_file_name_f = f'{mol_filename}_{distance1:.3f}.mol'
    #Chem.MolToMolFile(new_mol_f, output_file_name_f)

    # Loop for the second translation direction
    for step2 in range(num_loops_second):

        # Copy the molecule after the first translation
        new_mol_s = Chem.Mol(new_mol_f)


        # Define fragments and bond atoms for the second translation direction
        atom1_index_s, atom2_index_s, fragment1_atoms_s, fragment2_atoms_s = define_fragments_second_direction(new_mol_s)

 
        # Calculate the vector along the bond (after the first translation)
        atom1_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom1_index_s)
        atom2_coords_s = new_mol_s.GetConformer().GetAtomPosition(atom2_index_s)
        bond_vector_s = np.array(atom2_coords_s) - np.array(atom1_coords_s)
        bond_length_s = np.linalg.norm(bond_vector_s)
        unit_bond_vector_s = bond_vector_s / bond_length_s

        starting_distance2 = bond_length_s 

        # Calculate the distance for this step
        distance2 = starting_distance2 + step_second * step2
        
        # Calculate the translation vector along the bond
        translation_vector_s = unit_bond_vector_s * abs(distance2 - starting_distance2)


        # Move fragment 1 along the bond
        for atom_index_s in fragment1_atoms_s:
            atom_position_s = new_mol_s.GetConformer().GetAtomPosition(atom_index_s)
            new_position_s = atom_position_s + translation_vector_s 
            new_mol_s.GetConformer().SetAtomPosition(atom_index_s, new_position_s)

        # Save the modified molecule coordinates with both distances in the filename
        output_file_name = f'{mol_filename}_{distance1:.3f}_{distance2:.3f}.mol'
        Chem.MolToMolFile(new_mol_s, output_file_name)
